## Costruzione del dataset contenente per ogni campione la media della TIC per ogni minuto

### Costruzione del dataframe

In [3]:
import pandas as pd
import pyopenms as pms
import matplotlib.pyplot as plt

#### Creazione un primo df di appoggio contenente origine - cultivar e percorso del file mzdata VOC del campione

In [12]:
tabella_file = pd.read_excel("C:\\Users\\atocc\Desktop\\annotazioni.xlsx")
path_base = "C:\\Users\\atocc\Desktop\\idrocarburi-mzdata\\"
elenco_origini = []
elenco_cultivar = []
elenco_percorsi = []
for indice_riga, riga in tabella_file.iterrows():
    if str(riga['Idrocarburi']).strip() == 'Sì':
        elenco_origini.append(str(riga['Origine']).strip().lower())
        elenco_cultivar.append(str(riga['CV']).strip().lower())
        elenco_percorsi.append(path_base + str(riga['Sample']).strip() + '.mzdata.xml')
df_file = pd.DataFrame({'Origine': elenco_origini, 'Cultivar': elenco_cultivar, 'File': elenco_percorsi})
df_file

,Origine,Cultivar,File
0,spagna,arbequina,C:\Users\atocc\Desktop\idrocarburi-mzdata\2093...
1,spagna,arbequina,C:\Users\atocc\Desktop\idrocarburi-mzdata\2163...
2,portogallo,arbequina,C:\Users\atocc\Desktop\idrocarburi-mzdata\2164...
3,portogallo,arbequina,C:\Users\atocc\Desktop\idrocarburi-mzdata\2164...
4,portogallo,arbequina,C:\Users\atocc\Desktop\idrocarburi-mzdata\2164...
...,...,...,...
210,grecia-creta,psilolia,C:\Users\atocc\Desktop\idrocarburi-mzdata\2134...
211,grecia-creta,psilolia,C:\Users\atocc\Desktop\idrocarburi-mzdata\2134...
212,grecia-creta,psilolia,C:\Users\atocc\Desktop\idrocarburi-mzdata\2134...
213,grecia-creta,psilolia,C:\Users\atocc\Desktop\idrocarburi-mzdata\2134...


#### Definizione di una funzione che per ogni experiment (corrispondente ad un file) mi calcola la corrente ionica totale (TIC) e calcola la media della TIC in ogni minuto nei primi 40 minuti

In [13]:
min_range = 1
max_range = 41
def calcolaMedie(exp):
    tic = exp.calculateTIC()
    retention_time, intensity = tic.get_peaks()
    lista_medie = []
    pos_t_i = 0
    for min in range(min_range,max_range):
        somma = 0
        cont = 0
        while retention_time[pos_t_i] < 60*min :
            somma += intensity[pos_t_i]
            pos_t_i += 1
            cont += 1
        mean_intensity = somma/cont
        lista_medie.append(mean_intensity)
    return lista_medie

### Costruzione del dataset di lavoro inserendo i target 'origine' e 'cultivar' e 40 feature corrispondenti alla corrente ionica totale media in ogni minuto per ogni campione

In [14]:
colonna_origini = []
colonna_cultivar = []
colonna_valori = []
for index, row in df_file.iterrows():
    print('Campione ' + str(index))
    colonna_origini.append(row['Origine'])
    colonna_cultivar.append(row['Cultivar'])
    exp = pms.MSExperiment()
    #savfil = pms.SavitzkyGolayFilter()
    #savfil.filterExperiment(exp)
    #bernFilt = pms.BernNorm()
    #bernFilt.filterPeakMap(exp)
    pms.MzDataFile().load(str(row.File), exp)
    colonna_valori.append(calcolaMedie(exp))
df = pd.DataFrame(colonna_valori, columns=['minuto {}'.format(i+1) for i in range(max_range-1)])
# Aggiungo la colonna cultivar
df.insert(0,'Cultivar', colonna_cultivar)
# Aggiungo la colonna origine
df.insert(0,'Origine', colonna_origini)

Campione 0
Campione 1
Campione 2
Campione 3
Campione 4
Campione 5
Campione 6
Campione 7
Campione 8
Campione 9
Campione 10
Campione 11
Campione 12
Campione 13
Campione 14
Campione 15
Campione 16
Campione 17
Campione 18
Campione 19
Campione 20
Campione 21
Campione 22
Campione 23
Campione 24
Campione 25
Campione 26
Campione 27
Campione 28
Campione 29
Campione 30
Campione 31
Campione 32
Campione 33
Campione 34
Campione 35
Campione 36
Campione 37
Campione 38
Campione 39
Campione 40
Campione 41
Campione 42
Campione 43
Campione 44
Campione 45
Campione 46
Campione 47
Campione 48
Campione 49
Campione 50
Campione 51
Campione 52
Campione 53
Campione 54
Campione 55
Campione 56
Campione 57
Campione 58
Campione 59
Campione 60
Campione 61
Campione 62
Campione 63
Campione 64
Campione 65
Campione 66
Campione 67
Campione 68
Campione 69
Campione 70
Campione 71
Campione 72
Campione 73
Campione 74
Campione 75
Campione 76
Campione 77
Campione 78
Campione 79
Campione 80
Campione 81
Campione 82
Campione 83
Ca

In [15]:
df

,Origine,Cultivar,minuto 1,minuto 2,minuto 3,minuto 4,minuto 5,minuto 6,minuto 7,minuto 8,...,minuto 31,minuto 32,minuto 33,minuto 34,minuto 35,minuto 36,minuto 37,minuto 38,minuto 39,minuto 40
0,spagna,arbequina,285552.657407,222528.687023,479480.820611,841350.133080,151055.500000,293072.973282,210358.992366,231720.041825,...,254848.003817,242908.756654,330858.515267,3.109311e+05,895092.545802,507366.452471,1.602090e+06,1.108234e+06,1.311983e+06,1.959019e+06
1,spagna,arbequina,350859.666667,275378.450382,371042.889313,613049.079848,205299.877863,401429.790076,220935.912214,431963.661597,...,384002.698473,501966.589354,452314.958015,2.748970e+06,741897.832061,762177.859316,1.046992e+06,1.043548e+06,1.529580e+06,2.195992e+06
2,portogallo,arbequina,344787.731481,267565.671756,370105.251908,594541.433460,201934.870229,506640.347328,272863.492366,350155.752852,...,304054.125954,504697.619772,433351.675573,2.069047e+06,705018.629771,735898.395437,1.093091e+06,1.066879e+06,1.558130e+06,2.260089e+06
3,portogallo,arbequina,338957.527778,263786.790076,382563.980916,602526.395437,198295.335878,468896.087786,263291.324427,412403.623574,...,352494.148855,512697.912548,448702.515267,1.423215e+06,725906.568702,813684.623574,9.590397e+05,1.070762e+06,1.441533e+06,2.200444e+06
4,portogallo,arbequina,335893.537037,259351.702290,370968.881679,595496.049430,195838.366412,445420.213740,241029.889313,304775.984791,...,303901.793893,430377.726236,388433.618321,1.575730e+06,579745.820611,713441.722433,1.234962e+06,1.037187e+06,1.409344e+06,2.097782e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,grecia-creta,psilolia,232529.898148,171412.179389,313677.156489,571485.133080,121033.053435,309337.423664,150019.377863,143070.973384,...,304278.450382,393241.589354,310216.419847,5.426645e+05,761789.545802,566713.517110,6.497911e+05,1.007910e+06,1.318856e+06,1.713064e+06
211,grecia-creta,psilolia,238565.101852,177041.629771,317997.790076,567086.901141,124256.751908,310766.561069,147907.110687,173930.475285,...,308148.633588,426888.711027,354341.389313,6.857394e+05,791463.072519,589462.034221,6.786467e+05,1.026454e+06,1.319617e+06,1.725690e+06
212,grecia-creta,psilolia,238005.898148,175727.824427,314296.770992,574093.931559,125273.282443,291120.496183,152215.454198,150483.212928,...,501606.057252,426067.456274,365275.515267,7.810261e+05,677070.080153,602479.749049,6.712382e+05,1.110826e+06,1.203354e+06,1.694784e+06
213,grecia-creta,psilolia,225468.259259,168102.087786,289200.973282,539879.425856,120456.423664,289190.587786,145751.194656,142537.197719,...,605190.992366,480501.585551,283547.358779,5.653339e+05,626207.706107,503311.205323,7.386943e+05,1.079414e+06,1.100055e+06,1.591723e+06


### Salvataggio del dataframe in un csv da utilizzare poi per testare i vari algoritmi di machine learning

In [16]:
df['Cultivar'].value_counts()

picual                  50
psilolia                43
koroneiki               31
arbequina               31
coratina                21
chemlali                 8
hojiblanca               8
manzanilla               5
morisca                  3
picudo                   3
moraiolo                 2
frantoio                 1
carolea                  1
blend                    1
leccio del corno         1
chemlali 90%             1
maurino                  1
neroliva                 1
ogliarola                1
carnenzuela              1
koroneiki & neroliva     1
Name: Cultivar, dtype: int64

In [17]:
df['Origine'].value_counts()

spagna                  92
grecia-creta            43
italia                  28
grecia - peloponneso    23
portogallo              10
grecia                  10
tunisia                  9
Name: Origine, dtype: int64

In [18]:
df.to_csv('../dataset/df_media_TIC_Idro_al_minuto.csv', index=False)